In [ ]:
import os
import json
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

# ========== [1] ChromeDriver Ayarları ==========

chrome_driver_path = r"C:\Users\emirhan.gul\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
options = Options()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--incognito") # Gizli modda açarak çerez sorunlarını azaltabiliriz

service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
})
driver.set_page_load_timeout(30) # Sayfa yükleme zaman aşımını artırdık

# ========== [2] Yardımcı Fonksiyonlar ==========

def tum_sayfalari_kaziyip_ilan_linklerini_topla(driver):
    base_url = "https://www.arabam.com/ikinci-el/otomobil/volkswagen-passat-1-6-tdi-bluemotion"
    total_pages = 43 # Belirtilen toplam sayfa sayısı
    ilan_link_set = set() # Tekrar eden linkleri engellemek için set kullanılıyor
    hata_sayfalar = []
    toplam_cekilen_ilan_sayisi = 0

    for page in range(1, total_pages + 1):
        url = f"{base_url}?take=50&page={page}" # Tüm sayfalar için 'page' parametresi kullanılıyor
        print(f"\n📄 ➤ Sayfa {page} / {total_pages}: {url}")

        try:
            driver.get(url)
            
            # Çerezleri kabul etme denemesi (Eğer varsa)
            try:
                accept_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
                )
                accept_button.click()
                print("🍪 Çerezler kabul edildi.")
                time.sleep(1) # Tıklamadan sonra kısa bir bekleme
            except (TimeoutException, NoSuchElementException):
                # Çerez pop-up'ı yoksa veya hemen görünmüyorsa devam et
                pass

            # İlan linklerinin yüklenmesini bekle, ancak timeout olursa da devam et
            try:
                # Ana ilanların yüklendiğini gösteren bir elemanı bekleyin
                # Örneğin, ilk ilan kartının görünmesini bekleyebiliriz
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "a.link-overlay, .listing-card"))
                )
                time.sleep(random.uniform(1.5, 3.0)) # İnsan benzeri bir gecikme
            except TimeoutException:
                print(f"⚠️ Sayfa {page}: İlan elemanları belirtilen sürede yüklenemedi. Mevcut HTML'den çekilmeye çalışılıyor.")
            except WebDriverException as e:
                print(f"❌ Sayfa {page}: WebDriver hatası oluştu: {e}")
                hata_sayfalar.append((page, str(e)))
                continue # Bu sayfayı atla ve bir sonraki sayfaya geç

            ilan_anchors = driver.find_elements(By.CSS_SELECTOR, "a.link-overlay")
            print(f"🔍 Sayfa {page}: HTML'de {len(ilan_anchors)} potansiyel <a> etiketi bulundu.")

            current_page_new_links = 0
            for anchor in ilan_anchors:
                href = anchor.get_attribute("href")
                # URL'yi standardize et: sonundaki '/' işaretini kaldır
                if href and href.startswith("https://www.arabam.com/ilan/"):
                    # URL'nin sonunda bir slash varsa kaldırıyoruz
                    if href.endswith('/'):
                        href = href[:-1]
                    
                    if href not in ilan_link_set:
                        ilan_link_set.add(href)
                        toplam_cekilen_ilan_sayisi += 1
                        current_page_new_links += 1
                        # Her ilan linkini yazdırmak yerine, belli aralıklarla veya yeni link sayısıyla ilgili bilgi verelim
                        # print(f"🔗 Sayfa {page}, Yeni İlan: {href}")

            print(f"✅ Sayfa {page} için {current_page_new_links} yeni benzersiz ilan linki eklendi.")


        except Exception as e:
            print(f"❌ HATA! Sayfa {page} işlenirken genel bir sorun oluştu: {e}")
            hata_sayfalar.append((page, str(e)))

        print("-" * 70)
        time.sleep(random.uniform(1.2, 2.4)) # Sayfalar arası daha uzun ve rastgele bir bekleme

    # Tüm linkleri içeren DataFrame oluştur ve CSV'ye yaz
    df = pd.DataFrame(list(ilan_link_set), columns=["ilan_linki"])
    df.to_csv("yeni_ilan_linkleri.csv", index=False, encoding="utf-8-sig")
    print(f"\n✅ TOPLAM {len(ilan_link_set)} benzersiz ilan linki 'yeni_ilan_linkleri.csv' dosyasına kaydedildi.")

    if hata_sayfalar:
        print("\n🚨 HATA RAPORU:")
        print(f"{len(hata_sayfalar)} sayfa hatalıydı:")
        for page, error in hata_sayfalar:
            print(f" - Sayfa {page}: {error}")
        print("\nBu sayfalar için tekrar deneme veya manuel kontrol gerekebilir.")

# ========== [3] Ana Çalıştırma Bloğu ==========
if __name__ == "__main__":
    try:
        tum_sayfalari_kaziyip_ilan_linklerini_topla(driver)
    except Exception as e:
        print(f"Kritik bir hata oluştu: {e}")
    finally:
        driver.quit() # Hata olsa bile tarayıcıyı kapat


📄 ➤ Sayfa 1 / 43: https://www.arabam.com/ikinci-el/otomobil/volkswagen-passat-1-6-tdi-bluemotion?take=50&page=1
🔍 Sayfa 1: HTML'de 100 potansiyel <a> etiketi bulundu.
✅ Sayfa 1 için 50 yeni benzersiz ilan linki eklendi.
----------------------------------------------------------------------

📄 ➤ Sayfa 2 / 43: https://www.arabam.com/ikinci-el/otomobil/volkswagen-passat-1-6-tdi-bluemotion?take=50&page=2
🔍 Sayfa 2: HTML'de 100 potansiyel <a> etiketi bulundu.
✅ Sayfa 2 için 49 yeni benzersiz ilan linki eklendi.
----------------------------------------------------------------------

📄 ➤ Sayfa 3 / 43: https://www.arabam.com/ikinci-el/otomobil/volkswagen-passat-1-6-tdi-bluemotion?take=50&page=3
🔍 Sayfa 3: HTML'de 100 potansiyel <a> etiketi bulundu.
✅ Sayfa 3 için 50 yeni benzersiz ilan linki eklendi.
----------------------------------------------------------------------

📄 ➤ Sayfa 4 / 43: https://www.arabam.com/ikinci-el/otomobil/volkswagen-passat-1-6-tdi-bluemotion?take=50&page=4
🔍 Sayfa 4: 